In [109]:
import datetime
import time
import sys
sys.path.append(r'/home/hengtian/lucy/factor-analyzer-v4/')
import pymysql
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import multiprocessing
import os
import queue
import tqdm
# from src import data 
# from src import analyzer
sys.path.append(os.path.abspath('/home/hengtian/kate/'))
import get_factor_tools as gft
# import datetime as dt
from imp import reload
reload(gft)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# import cufflinks as cf
# cf.go_offline()

sys.path.append(r'/home/hengtian/lucy/PortfolioOptimizer/')
from PortfolioOptimizerMaster.src import tools

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression

In [16]:
def multi_process(stock_list, func,pro_num):
    q = multiprocessing.Manager().Queue()
    param = []
    for stock in stock_list:
        param.append((stock,q))
    # print(param)
    #task
    with multiprocessing.Pool(processes =pro_num) as pool:
        pool.starmap(func,param)

    #extract
    elements = []
    while q.qsize():
        elements.append(q.get())
    return elements

In [ ]:
# 每日最高价
h_post = gft.get_stock_factor('highest_price_post',start_date =datetime.date(2013, 1, 1),end_date = datetime.date(2021, 8, 9))
h_post.to_csv('highest_price_post_0809.csv')
#每日最低价
l_post =gft.get_stock_factor('lowest_price_post',start_date =datetime.date(2013, 1, 1),end_date = datetime.date(2021, 8, 9))
l_post.to_csv('lowest_price_post_0809.csv')

In [ ]:
h_post = pd.read_csv('highest_price_post_0809.csv',index_col =0)
l_post = pd.read_csv('lowest_price_post_0809.csv',index_col =0)

In [ ]:
#成交金额
amt = pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/turnover_value.csv",index_col = 0)
amt = amt['2013':'2021-08-09']
# amt = amt[ovlp]

In [ ]:
close_price = pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/close_price_post.csv",index_col = 0)
close_price = close_price['2013':'2021-08-09']
close_prie= close_price.replace(0,np.nan)

In [ ]:
op_price = pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/open_price_post.csv",index_col = 0)
op_price= op_price.replace(0,np.nan)

In [ ]:
#涨跌幅
n_pct = close_price/close_price.shift(1)-1

In [ ]:
#最高涨幅
h_post.index=close_price.index
max_pct =h_post/close_price.shift(1)-1

In [ ]:
# 当天最高价/20日内最低价
min_20 = l_post.rolling(20,min_periods = 10).min()
high_vs_20 = h_post/min_20.shift(1)

In [ ]:
high_vs_20.index=close_price.index

In [ ]:
#40日avg
forty_avg = close_price.rolling(40,min_periods =20).mean()
#收盘价大于40日均线
close_vs_forty= close_price/forty_avg.shift(1)

In [ ]:
#当日和前5日均量
amt= amt.replace(0,np.nan)
amt = amt.shift(-1)
amt_5_avg = amt.rolling(5,min_periods=3).mean()
amt_past5 = amt/amt_5_avg.shift(1)

In [ ]:
# 当天收盘vs当天开盘
in_day = close_price/op_price

In [ ]:
comp_dict ={
    'max_pct':max_pct,
    'close_vs_forty':close_vs_forty,
    'high_vs_20':high_vs_20,
    'amt_past5':amt_past5,
    'in_day':in_day
}

In [ ]:
"""
结合所有的特征
"""
og = n_pct.unstack().rename('n_pct')
df = pd.DataFrame(og).reset_index()
df.columns = ['stock_code','date','n_pct']
for comp_name,comp_df in comp_dict.items():
    temp =comp_df.unstack().rename(comp_name)
    temp.index.names = ['stock_code','date']
#     print(temp)
    df = pd.merge(df,temp.reset_index(),how = 'left',on = ['stock_code','date'])
#     print(df)
#     break

In [ ]:
"""
选取所有满足标志k线的股票&日期
"""
df.loc[(df['n_pct']>0.02)&(df['max_pct']>0.03)&(df['in_day']>1)&(df['close_vs_forty']>1)&(df['amt_past5']>0.6)&(df['high_vs_20']>1.2), 'is_k'] = 1

In [ ]:
k_line =df[df.k_line==1]

In [17]:
factors =pd.read_csv('factors.csv')

In [20]:
is_k =pd.pivot_table(factors,values = 'k_line',index='date',columns = 'stock_code',dropna= True).sort_index()

In [25]:
intval =pd.pivot_table(factors,values = 'interval_days',index='date',columns = 'stock_code',dropna= True).sort_index()

In [78]:
"""
针对is_k的dataframe，找到间隔日期
"""
def find_kday(stock,q):
    temp_f =is_k[stock]
    temp_s = temp_f[temp_f==1].reset_index()
    if len(temp_s)>0:
        sig = []
        for date in list(temp_f.index):
            try:
                sig_day = temp_s[temp_s.date<=date].iloc[-1].values[0]
    #             print(sig_day)
                intval = len(is_k.loc[sig_day:date])-1
                sig.append(intval)
            except IndexError:
                sig.append(np.nan)
        sig = pd.DataFrame(sig,index = is_k.index,columns = [stock])
    else:
        sig =pd.DataFrame(np.nan, index=is_k.index, columns=[stock])
    return q.put({stock:sig})
#     return sig

In [79]:
elements = multi_process(is_k.columns,find_kday,20)
elmts =[]
for i in range(len(elements)):
    stock = list(elements[i].keys())[0]
    exp = elements[i][stock]
    elmts.append(exp)
intval_days = pd.concat(elmts,axis =1)

In [113]:
"""
当天距离标志K线的最高涨幅（区间最高价/标志K线收盘价）
需要is_k & k_close_price_post & h_post  三个dataframe
"""

def find_max_chg(stock,q):
    temp = is_k[is_k.stock_code==stock]
    temp_k =temp[temp==1].reset_index()
    k_price = k_close_price_post[stock]
    if len(temp_k)>0:
        begin =list(temp_k.date)[0]
        temp_max =[]
        for date in temp.date:
            if date<begin:
                temp_max.append(np.nan)
            else:
                kday =temp_k[temp_k.date<=date].iloc[-1].values[0]
                if date==kday:
                    temp_max.append(np.nan)
                else:
                    nxt = list(h_post.index).index(kday)+1
                    nxtday =list(h_post.index)[nxt]
                    t_max =h_post[stock].loc[nxtday:date].max()/k_price.loc[nxtday].values
                    temp_max.append(t_max)
        temp_max = pd.DataFrame(temp_max,index=is_k.index, columns=[stock])
    else:
        temp_max =pd.DataFrame(np.nan, index=is_k.index, columns=[stock])
    return q.put({stock:temp_max})

In [ ]:
elements = multi_process(is_k.columns,find_max_chg,30)

In [ ]:
elmts =[]
for i in range(len(elements)):
    stock = list(elements[i].keys())[0]
    exp = elements[i][stock]
    t_df =pd.DataFrame(exp)
#     print(t_df)
    elmts.append(t_df)
max_chg =pd.concat(elmts,axis =1)

In [ ]:
"""
当天距离标志K线的最低涨幅（区间最低价/标志K线收盘价）
"""
def find_min_chg(stock,q):
    temp = is_k[is_k.stock_code==stock]
    temp_k =temp[temp==1].reset_index()
    k_price = k_close_price_post[stock]
    if len(temp_k)>0:
        begin =list(temp_k.date)[0]
        temp_max =[]
        for date in temp.date:
            if date<begin:
                temp_min.append(np.nan)
            else:
                kday =temp_k[temp_k.date<=date].iloc[-1].values[1]
                if date==kday:
                    temp_min.append(np.nan)
                else:
                    nxt = list(l_post.index).index(kday)+1
                    nxtday =list(l_post.index)[nxt]
                    t_min =l_post[stock].loc[nxtday:date].min()/k_price.loc[nxtday].values
                    temp_min.append(t_min)
        temp_min = pd.DataFrame(temp_min,index=is_k.index, columns=[stock])
    else:
        temp_min =pd.DataFrame(np.nan, index=is_k.index, columns=[stock])
    return q.put({stock:temp_min})

In [ ]:
elements = multi_process(set(is_k.columns),find_min_chg,30)

In [ ]:
elmts =[]
for i in range(len(elements)):
    stock = list(elements[i].keys())[0]
    exp = elements[i][stock]
    t_df =pd.DataFrame(exp)
#     print(t_df)
    elmts.append(t_df)
min_chg =pd.concat(elmts,axis =1)

# 标志K线收盘价和20、40均线乖离率

In [ ]:
avg20 =close_price.rolling(20,min_periods =10).mean()
avg20 = avg20.shift(1)
avg40 = close_price.rolling(40,min_periods =20).mean()
avg40 =avg40.shift(1)

In [ ]:
avg20 = avg20.unstack().reset_index()
avg20.columns =['stock_code','date','avg_20']
avg40 = avg40.unstack().reset_index()
avg40.columns =['stock_code','date','avg_40']

In [ ]:
k_line1 = k_line.merge(avg20,how = 'left',on =['stock_code','date'])
k_line1 = k_line1.merge(avg40,how = 'left',on =['stock_code','date'])

In [ ]:
k_line1['20_dev'] =k_line1['k_close_price']/k_line1['avg_20']
k_line1['40_dev'] =k_line1['k_close_price']/k_line1['avg_40']
k_line1

In [ ]:
def fill_fwd(stock,q):
    temp =dfv5[dfv5.stock_code ==stock]
    temp_k =k_line1[k_line1.stock_code==stock]
    cbn =temp.merge(temp_k[['stock_code','date','avg_20','avg_40','20_dev','40_dev']],how ='left',on =['stock_code','date'])
    cbn['avg_20'].fillna(method='ffill',inplace = True)
    cbn['avg_40'].fillna(method='ffill',inplace = True)
    cbn['20_dev'].fillna(method='ffill',inplace = True)
    cbn['40_dev'].fillna(method='ffill',inplace = True)
    return q.put({stock:cbn})

In [ ]:
elements = multi_process(set(k_line.stock_code),fill_fwd,30)

In [ ]:
elmts =[]
for i in range(len(elements)):
    stock = list(elements[i].keys())[0]
    exp = elements[i][stock]
    elmts.append(exp)

In [ ]:
test = pd.concat(elmts)

In [ ]:
test[test.stock_code=='000001.XSHE'].head(50)

In [ ]:
dfv6 = dfv5.merge(test[['stock_code','date','20_dev','40_dev']],how = 'left',on = ['stock_code','date'])

# 标志K线均线变异系数

In [ ]:
std_5 = close_price.rolling(5,min_periods =3).std()
std_5 = std_5.shift(1)
std_10 = close_price.rolling(10,min_periods =5).std()
std_10 = std_10.shift(1)
std_20 = close_price.rolling(20,min_periods =10).std()
std_20 = std_20.shift(1)
std_40=close_price.rolling(40,min_periods =20).std()
std_40 = std_40.shift(1)
std_60= close_price.rolling(60,min_periods =30).std()
std_60 = std_60.shift(1)

In [ ]:
mean_5 = close_price.rolling(5,min_periods =3).mean()
mean_5 = mean_5.shift(1)
mean_10 = close_price.rolling(10,min_periods =5).mean()
mean_10 = mean_10.shift(1)
mean_20 = close_price.rolling(20,min_periods =10).mean()
mean_20 = mean_20.shift(1)
mean_40=close_price.rolling(40,min_periods =20).mean()
mean_40 = mean_40.shift(1)
mean_60= close_price.rolling(60,min_periods =30).mean()
mean_60 = mean_60.shift(1)

In [ ]:
coeff_dev5 =std_5/mean_5

In [ ]:
coeff_dev10 =std_10/mean_10

In [ ]:
coeff_dev20 =std_20/mean_20

In [ ]:
coeff_dev40 =std_40/mean_40

In [ ]:
coeff_dev60 =std_60/mean_60

In [ ]:
coeff_dev5 = coeff_dev5.unstack().reset_index()
coeff_dev5.columns =['stock_code','date','coeff_dev5']

In [ ]:
data =coeff_dev5.copy()
comp = {
    'coeff_dev10':coeff_dev10,
    'coeff_dev20':coeff_dev20,
    'coeff_dev40':coeff_dev40,
    'coeff_dev60':coeff_dev60,
}

In [ ]:
for comp_name,comp_df in comp.items():
    temp =comp_df.unstack().rename(comp_name)
    temp.index.names = ['stock_code','date']
#     print(temp)
    data = pd.merge(data,temp.reset_index(),how = 'left',on = ['stock_code','date'])

In [ ]:
k_line2 = k_line1.merge(data[['stock_code','date','coeff_dev5','coeff_dev10','coeff_dev20','coeff_dev40','coeff_dev60']],how = 'left',on = ['stock_code','date'])

In [ ]:
dfv6 = pd.read_csv('oval_dataframe.csv',index_col = 0)
k_line2 = pd.read_csv('k_line_ftr.csv',index_col =0)

In [12]:
"""
根据每只股票fill forward
"""
def ffill_stock(stock,q):
    temp = dfv6[dfv6.stock_code==stock]
    att = k_line2[k_line2.stock_code==stock]
    s_df = temp.merge(att[['stock_code','date','k_coeff']],how = 'left',on = ['stock_code','date'])
    s_df['k_coeff'].fillna(method = 'ffill',inplace =True)
    return q.put({stock:s_df})

In [ ]:
elements = multi_process(set(k_line2.stock_code),ffill_stock,30)

In [ ]:
elmts =[]
for i in range(len(elements)):
    stock = list(elements[i].keys())[0]
    exp = elements[i][stock]
    elmts.append(exp)

In [ ]:
dfv10 = pd.concat(elmts)

In [ ]:
dfv10.to_csv('oval_dataframe.csv')
# k_line2.to_csv('k_line_ftr.csv')

In [ ]:
k_line2['k_coeff'] = k_line2[['coeff_dev5','coeff_dev10','coeff_dev20','coeff_dev40','coeff_dev60']].std(axis =1,skipna = False)/ k_line2[['coeff_dev5','coeff_dev10','coeff_dev20','coeff_dev40','coeff_dev60']].mean(axis =1,skipna = False)

In [ ]:
k_line2.drop('k_coeff',axis =1,inplace = True)

In [ ]:
factors = dfv10[['stock_code','date','k_line','interval_days','k_chg','max_chg','min_chg','20_dev','40_dev','k_coeff']]

In [ ]:
factors.to_csv('factors.csv')